# Load dependencies

In [ ]:
using Pkg
Pkg.activate(@__DIR__)

In [ ]:
env = Pkg.Types.EnvCache(joinpath(@__DIR__, "Project.toml"))
if !Pkg.Operations.is_instantiated(env)
    print("Instantiating environment... ")
    # Needed while the package is not registered
    package_dir = joinpath(@__DIR__, "..", "..")
    Pkg.develop(path=package_dir)
    # The "develop" command will automatically instantiate the environment
    # Pkg.instantiate()
    # Current dependencies
    # Pkg.add(["Plots", "JuMP", "HiGHS", "JSON", "Distributions"])
end


In [ ]:
using Plots
using JSON
using JuMP
using HiGHS
#
using Statistics
using Random

## Read input data

In [ ]:
data = JSON.parsefile(joinpath(@__DIR__, "30_2017-01-01.json"))

In [ ]:
data["Parameters"]

In [ ]:
@show T = data["Parameters"]["Time horizon (h)"]
demand = zeros(T)
for ts in values(data["Buses"])
    load = ts["Load (MW)"]
    if length(load) > 1
        demand .+= load
    end
end
plot(demand, linewidth=2, title="Demand", xlabel="Time", ylabel="MW")


In [ ]:
# Check the Reserves' data structure
for res in values(data["Reserves"])
    @assert res["Type"] == "Spinning"
    @assert length(res["Amount (MW)"]) == T
end

In [ ]:
single_reserve = zeros(T)
for ts in values(data["Reserves"])
    res = ts["Amount (MW)"]
    single_reserve .+= res
end
plot(single_reserve, linewidth=2, title="Reserve", xlabel="Time", ylabel="MW")

In [ ]:
generator_names = collect(keys(data["Generators"]))

# Regular (deterministic) UC

In [ ]:
uc = Model(HiGHS.Optimizer)

# Define variables

# deficit
@variable(uc, 0 <= deficit[t in 1:T])
# plant on/off
@variable(uc, 0 <= x[i in keys(data["Generators"]), t in 0:T] <= 1, Bin)
# startup
@variable(uc, 0 <= y[i in keys(data["Generators"]), t in 1:T] <= 1)
# generator output
@variable(uc, 0 <= g[i in keys(data["Generators"]), t in 0:T]
    <= data["Generators"][i]["Production cost curve (MW)"][end])

# Define constraints

# load balance
@constraint(uc, LoadBalance[t in 1:T], sum(g[i, t] for i in keys(data["Generators"])) + deficit[t] == demand[t])

# initial state
@constraint(uc, [i in keys(data["Generators"])],
    g[i, 0] == data["Generators"][i]["Initial power (MW)"])
@constraint(uc, [i in keys(data["Generators"])],
    x[i, 0] == ifelse(data["Generators"][i]["Initial power (MW)"] > 0, 1, 0))
# ramp limit
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    g[i, t] - g[i, t-1] <= data["Generators"][i]["Ramp up limit (MW)"])
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    g[i, t-1] - g[i, t] <= data["Generators"][i]["Ramp down limit (MW)"])
# startup and shutdown
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    g[i, t]
    <= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][end])
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    g[i, t] >= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][1])
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    x[i, t] - x[i, t-1] <= y[i, t])

# Define objective

@objective(uc, Min,
    # generator variable costs
    sum(
        (data["Generators"][i]["Production cost curve (\$)"][end] /
            data["Generators"][i]["Production cost curve (MW)"][end])
        * g[i, t] for i in keys(data["Generators"]), t in 0:T)
    # startup costs
    + sum(
        data["Generators"][i]["Startup costs (\$)"][1]
        * y[i, t] for i in keys(data["Generators"]), t in 1:T)
    # deficit costs
    + sum(
        deficit[t] * data["Parameters"]["Power balance penalty (\$/MW)"] for t in 1:T)
)
# set_silent(uc)
optimize!(uc)
det_obj = objective_value(uc)

In [ ]:
uc

In [ ]:
det_x = value.(uc[:x])
det_g = Matrix(value.(uc[:g]))
det_d = value.(uc[:deficit])
p = plot(title="Generation / Deficit", xlabel="Hours", ylabel="Power (MW)")
plot!(p, det_g', linewidth=2, labels=reshape(generator_names, 1, :))
plot!(p, det_d, linewidth=4, labels="deficit")

# Generate scenarios

In [ ]:
S = 20
rng = Random.MersenneTwister(0)
range = 3.0
base_demand = deepcopy(demand)
scenario_demand = zeros(T, S)
for s in 1:S
    for t in 1:T
        scenario_demand[t, s] = max(0, base_demand[t] + (range * single_reserve[t] * randn(rng)))
    end
end
plot(scenario_demand, linewidth=2, title="$S demand scenarios", xlabel="Time", ylabel="MW", legend=:false)

# Standard stochastic (SAA) UC

In [ ]:
uc_saa = Model(HiGHS.Optimizer)

# Define variables

# deficit
@variable(uc_saa, 0 <= deficit[t in 1:T, s in 1:S])
# plant on/off
@variable(uc_saa, 0 <= x[i in keys(data["Generators"]), t in 0:T] <= 1, Bin)
# startup
@variable(uc_saa, 0 <= y[i in keys(data["Generators"]), t in 1:T] <= 1)
# generator output
@variable(uc_saa, 0 <= g[i in keys(data["Generators"]), t in 0:T, s in 1:S]
    <= data["Generators"][i]["Production cost curve (MW)"][end])

# Define constraints

# load balance
@constraint(uc_saa, LoadBalance[t in 1:T, s in 1:S],
    sum(g[i, t, s] for i in keys(data["Generators"])) + deficit[t, s] == scenario_demand[t, s])

# initial state
@constraint(uc_saa, [i in keys(data["Generators"]), s in 1:S],
    g[i, 0, s] == data["Generators"][i]["Initial power (MW)"])
@constraint(uc_saa, [i in keys(data["Generators"])],
    x[i, 0] == ifelse(data["Generators"][i]["Initial power (MW)"] > 0, 1, 0))
# ramp limit
@constraint(uc_saa, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t, s] - g[i, t-1, s] <= data["Generators"][i]["Ramp up limit (MW)"])
@constraint(uc_saa, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t-1, s] - g[i, t, s] <= data["Generators"][i]["Ramp down limit (MW)"])
# startup and shutdown
@constraint(uc_saa, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t, s]
    <= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][end])
@constraint(uc_saa, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t, s] >= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][1])
@constraint(uc_saa, [i in keys(data["Generators"]), t in 1:T],
    x[i, t] - x[i, t-1] <= y[i, t])

# Define objective

@objective(uc_saa, Min,
    # generator variable costs
    (1/S) * sum(
        (data["Generators"][i]["Production cost curve (\$)"][end] /
            data["Generators"][i]["Production cost curve (MW)"][end])
        * g[i, t, s] for i in keys(data["Generators"]), t in 0:T, s in 1:S)
    # startup costs
    + sum(
        data["Generators"][i]["Startup costs (\$)"][1]
        * y[i, t] for i in keys(data["Generators"]), t in 1:T)
    # deficit costs
    + (1/S) * sum(
        deficit[t, s] * data["Parameters"]["Power balance penalty (\$/MW)"] for t in 1:T, s in 1:S)
)
# set_silent(uc_saa)
set_optimizer_attribute(uc_saa, "mip_rel_gap", 0.01)
set_time_limit_sec(uc_saa, 300)
optimize!(uc_saa)
saa_obj = objective_value(uc_saa)

In [ ]:
saa_x = value.(uc_saa[:x])
saa_g = mean(Array{Float64, 3}(value.(uc_saa[:g])), dims=3)[:,:,1]
saa_d = mean(Array{Float64, 2}(value.(uc_saa[:deficit])), dims=2)[:,1]
p = plot(title="Generation / Deficit", xlabel="Hours", ylabel="Power (MW)")
plot!(p, saa_g', linewidth=2, labels=reshape(generator_names, 1, :))
plot!(p, saa_d, linewidth=4, labels="deficit")

# Affine UC

In [ ]:
uc_ldr = Model(HiGHS.Optimizer)

poly_size = 1
max_lag = max(1, 3)

# Define variables

# deficit
@variable(uc_ldr, 0 <= deficit[t in 1:T, s in 1:S])
# plant on/off
@variable(uc_ldr, 0 <= x[i in keys(data["Generators"]), t in 0:T] <= 1, Bin)
# startup
@variable(uc_ldr, 0 <= y[i in keys(data["Generators"]), t in 1:T] <= 1)
# generator output
@variable(uc_ldr, 0 <= g[i in keys(data["Generators"]), t in 0:T, s in 1:S]
    <= data["Generators"][i]["Production cost curve (MW)"][end])

# LDR coefs
@variable(uc_ldr, rule_g[i in keys(data["Generators"]), t in 0:T, lag in 1:min(t, T), p in 0:poly_size])
@constraint(uc_ldr, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t, s] == sum(rule_g[i, t, lag, p] * (scenario_demand[t + 1 - lag, s])^p for p in 0:poly_size, lag in 1:min(t, max_lag)))

# Define constraints

# load balance
@constraint(uc_ldr, LoadBalance[t in 1:T, s in 1:S],
    sum(g[i, t, s] for i in keys(data["Generators"])) + deficit[t, s] == scenario_demand[t, s])

# initial state
@constraint(uc_ldr, [i in keys(data["Generators"]), s in 1:S],
    g[i, 0, s] == data["Generators"][i]["Initial power (MW)"])
@constraint(uc_ldr, [i in keys(data["Generators"])],
    x[i, 0] == ifelse(data["Generators"][i]["Initial power (MW)"] > 0, 1, 0))
# ramp limit
@constraint(uc_ldr, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t, s] - g[i, t-1, s] <= data["Generators"][i]["Ramp up limit (MW)"])
@constraint(uc_ldr, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t-1, s] - g[i, t, s] <= data["Generators"][i]["Ramp down limit (MW)"])
# startup and shutdown
@constraint(uc_ldr, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t, s]
    <= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][end])
@constraint(uc_ldr, [i in keys(data["Generators"]), t in 1:T, s in 1:S],
    g[i, t, s] >= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][1])
@constraint(uc_ldr, [i in keys(data["Generators"]), t in 1:T],
    x[i, t] - x[i, t-1] <= y[i, t])

# Define objective

@objective(uc_ldr, Min,
    # generator variable costs
    (1/S) * sum(
        (data["Generators"][i]["Production cost curve (\$)"][end] /
            data["Generators"][i]["Production cost curve (MW)"][end])
        * g[i, t, s] for i in keys(data["Generators"]), t in 0:T, s in 1:S)
    # startup costs
    + sum(
        data["Generators"][i]["Startup costs (\$)"][1]
        * y[i, t] for i in keys(data["Generators"]), t in 1:T)
    # deficit costs
    + (1/S) * sum(
        deficit[t, s] * data["Parameters"]["Power balance penalty (\$/MW)"] for t in 1:T, s in 1:S)
)
# set_silent(uc_ldr)
set_optimizer_attribute(uc_ldr, "mip_rel_gap", 0.01)
set_time_limit_sec(uc_ldr, 300)
optimize!(uc_ldr)
ldr_obj = objective_value(uc_ldr)

In [ ]:
ldr_rule_g = value.(uc_ldr[:rule_g])

ldr_x = value.(uc_ldr[:x])
ldr_g = mean(Array{Float64, 3}(value.(uc_ldr[:g])), dims=3)[:,:,1]
ldr_d = mean(Array{Float64, 2}(value.(uc_ldr[:deficit])), dims=2)[:,1]
p = plot(title="Generation / Deficit", xlabel="Hours", ylabel="Power (MW)")
plot!(p, ldr_g', linewidth=2, labels=reshape(generator_names, 1, :))
plot!(p, ldr_d, linewidth=4, labels="deficit")

# Polyhedral LDR UC

In [ ]:
using Distributions
using LinearDecisionRules
LDR = LinearDecisionRules;

In [ ]:
# Demand uncertainty for the LDR
range = 3.0
sigmas = 2.5
ampl_var = deepcopy(single_reserve) * range
var_demand = Normal.(0.0, ampl_var)
var_demand = truncated.(var_demand, -sigmas * ampl_var, sigmas * ampl_var)

base_demand = deepcopy(demand)
plot(title="Demand band of $(sigmas)σ, with σ = $(range) × reserve, ", xlabel="Time", ylabel="MW")
plot!(demand, linewidth=2, label="Demand")
plot!(demand .+ sigmas * ampl_var, linewidth=1, label="Upper Demand")
plot!(demand .- sigmas * ampl_var, linewidth=1, label="Lower Demand")

In [ ]:
uc_ldr_p = LDR.LDRModel(HiGHS.Optimizer)

# Define variables

# deficit
@variable(uc_ldr_p, 0 <= deficit[t in 1:T])
# plant on/off
@variable(uc_ldr_p, 0 <= x[i in keys(data["Generators"]), t in 0:T] <= 1, LDR.FirstStage, integer=true)
# startup
@variable(uc_ldr_p, 0 <= y[i in keys(data["Generators"]), t in 1:T] <= 1, LDR.FirstStage, integer=true)
# generator output
@variable(uc_ldr_p, 0 <= g[i in keys(data["Generators"]), t in 0:T]
    <= data["Generators"][i]["Production cost curve (MW)"][end])

# variable Demand
@variable(uc_ldr_p, demand_extra[t in 1:T] in LDR.Uncertainty(distribution=var_demand[t]))

# Define constraints

# load balance
@constraint(uc_ldr_p, LoadBalance[t in 1:T], sum(g[i, t] for i in keys(data["Generators"])) + deficit[t] == demand[t] + demand_extra[t])

# initial state
@constraint(uc_ldr_p, [i in keys(data["Generators"])],
    g[i, 0] == data["Generators"][i]["Initial power (MW)"])
@constraint(uc_ldr_p, [i in keys(data["Generators"])],
    x[i, 0] == ifelse(data["Generators"][i]["Initial power (MW)"] > 0, 1, 0))
# ramp limit
@constraint(uc_ldr_p, [i in keys(data["Generators"]), t in 1:T],
    g[i, t] - g[i, t-1] <= data["Generators"][i]["Ramp up limit (MW)"])
@constraint(uc_ldr_p, [i in keys(data["Generators"]), t in 1:T],
    g[i, t-1] - g[i, t] <= data["Generators"][i]["Ramp down limit (MW)"])
# startup and shutdown
@constraint(uc_ldr_p, [i in keys(data["Generators"]), t in 1:T],
    g[i, t]
    <= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][end])
@constraint(uc_ldr_p, [i in keys(data["Generators"]), t in 1:T],
    g[i, t] >= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][1])
@constraint(uc_ldr_p, [i in keys(data["Generators"]), t in 1:T],
    x[i, t] - x[i, t-1] <= y[i, t])

# Define objective

@objective(uc_ldr_p, Min,
    # generator variable costs
    sum(
        (data["Generators"][i]["Production cost curve (\$)"][end] /
            data["Generators"][i]["Production cost curve (MW)"][end])
        * g[i, t] for i in keys(data["Generators"]), t in 0:T)
    # startup costs
    + sum(
        data["Generators"][i]["Startup costs (\$)"][1]
        * y[i, t] for i in keys(data["Generators"]), t in 1:T)
    # deficit costs
    + sum(
        deficit[t] * data["Parameters"]["Power balance penalty (\$/MW)"] for t in 1:T)
)
# set_silent(uc_ldr_p)
set_attribute(uc_ldr_p, LDR.SolveDual(), false)
optimize!(uc_ldr_p)
ldr_p_obj = objective_value(uc_ldr_p)

In [ ]:
ldr_p_x = LDR.get_decision.(uc_ldr_p, uc_ldr_p[:x])
ldr_p_g = Matrix(LDR.get_decision.(uc_ldr_p, uc_ldr_p[:g]))
ldr_p_d = LDR.get_decision.(uc_ldr_p, uc_ldr_p[:deficit])
p = plot(title="LDR decision", xlabel="Hours", ylabel="Power (MW)")
plot!(p, ldr_p_g', linewidth=2, label=reshape(generator_names, 1, :))
plot!(p, ldr_p_d, linewidth=4, label="Deficit")

# Out-of-sample tests

In [ ]:
function run_scenario(_x, _demand)

uc = Model(HiGHS.Optimizer)

# Define variables

# deficit
@variable(uc, 0 <= deficit[t in 1:T])
# plant on/off
@variable(uc, x[i in keys(data["Generators"]), t in 0:T] == _x[i, t])
# startup
@variable(uc, 0 <= y[i in keys(data["Generators"]), t in 1:T] <= 1)
# generator output
@variable(uc, 0 <= g[i in keys(data["Generators"]), t in 0:T]
    <= data["Generators"][i]["Production cost curve (MW)"][end])

# Define constraints

# load balance
@constraint(uc, LoadBalance[t in 1:T],
    sum(g[i, t] for i in keys(data["Generators"])) + deficit[t] == _demand[t])

# initial state
@constraint(uc, [i in keys(data["Generators"])],
    g[i, 0] == data["Generators"][i]["Initial power (MW)"])
@constraint(uc, [i in keys(data["Generators"])],
    x[i, 0] == ifelse(data["Generators"][i]["Initial power (MW)"] > 0, 1, 0))
# ramp limit
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    g[i, t] - g[i, t-1] <= data["Generators"][i]["Ramp up limit (MW)"])
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    g[i, t-1] - g[i, t] <= data["Generators"][i]["Ramp down limit (MW)"])
# startup and shutdown
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    g[i, t]
    <= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][end])
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    g[i, t] >= x[i, t] * data["Generators"][i]["Production cost curve (MW)"][1])
@constraint(uc, [i in keys(data["Generators"]), t in 1:T],
    x[i, t] - x[i, t-1] <= y[i, t])

# Define objective

@objective(uc, Min,
    # generator variable costs
    sum(
        (data["Generators"][i]["Production cost curve (\$)"][end] /
            data["Generators"][i]["Production cost curve (MW)"][end])
        * g[i, t] for i in keys(data["Generators"]), t in 0:T)
    # startup costs
    + sum(
        data["Generators"][i]["Startup costs (\$)"][1]
        * y[i, t] for i in keys(data["Generators"]), t in 1:T)
    # deficit costs
    + sum(
        deficit[t] * data["Parameters"]["Power balance penalty (\$/MW)"] for t in 1:T)
)

set_silent(uc)
optimize!(uc)

return objective_value(uc)
end

In [ ]:
rng = Random.MersenneTwister(123)
S_out = 50
range = 3.0
base_demand = deepcopy(demand)
scenario_demand_out = zeros(T, S_out)
for s in 1:S_out
    for t in 1:T
        scenario_demand_out[t, s] = max(0, base_demand[t] + (range * single_reserve[t] * randn(rng)))
    end
end
plot(scenario_demand_out, linewidth=2, title="$(S_out) demand curves", xlabel="Time", ylabel="MW", legend=:false)

In [ ]:
for s in 1:50
    if all(scenario_demand_out[:, s] .< demand .+ ampl_var * sigmas) &&
        all(scenario_demand_out[:, s] .> demand .- ampl_var * sigmas)
        continue
    else
        println("Scenario $s goes out of support")
    end
end

In [ ]:
@show det_obj
@show saa_obj
@show ldr_obj
@show ldr_p_obj
;

In [ ]:
det_v = 0.0
@time for s in 1:S_out
    det_v += run_scenario(det_x, scenario_demand_out[:, s]) / S_out
end
@show det_v
saa_v = 0.0
for s in 1:S_out
    saa_v += run_scenario(saa_x, scenario_demand_out[:, s]) / S_out
end
@show saa_v
ldr_v = 0.0
for s in 1:S_out
    ldr_v += run_scenario(ldr_x, scenario_demand_out[:, s]) / S_out
end
@show ldr_v
ldr_p_v = 0.0
for s in 1:S_out
    ldr_p_v += run_scenario(ldr_p_x, scenario_demand_out[:, s]) / S_out
end
@show ldr_p_v
;

In [ ]:
function apply_rule(_rule_g, _demand)
    return [
        sum(_rule_g[i, t, lag, p] * (_demand[t + 1 - lag])^p for p in 0:poly_size, lag in 1:min(t, max_lag))
        for i in keys(data["Generators"]), t in 1:T
    ]
end

In [ ]:
for s in 1:S_out
    print(s, " -> ")
    new_g = apply_rule(ldr_rule_g, scenario_demand_out[:, s])
    neg_gen = - clamp.(new_g, -Inf, 0.0)
    if sum(neg_gen) > 1e-8
        #println("  Found negative generation")
        #@show neg_gen
        @show sum(neg_gen, dims=1)
    else
        println()
    end
end